# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
*`education` — уровень образования клиента;
*`education_id` — идентификатор уровня образования;
*`family_status` — семейное положение;
*`family_status_id` — идентификатор семейного положения;
*`gender` — пол клиента;
*`income_type` — тип занятости;
*`debt` — имел ли задолженность по возврату кредитов;
*`total_income` — ежемесячный доход;
*`purpose` — цель получения кредита.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd # подключение необходимых библиотек
from pymystem3 import Mystem
from copy import deepcopy
m = Mystem()

data = pd.read_csv('/datasets/data.csv') # чтение файла с данными и сохранение в data
data.head(10) # получение первых 10 строк таблицы data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим общую информацию по таблице

In [2]:
data.info() # получение общей информации о данных в таблице data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Посчитаем кол-во пропусков и выведем уникальные значения для определения дальнейшних планов по проекту

In [3]:
data[data['days_employed'].isna()].head() # проверка таблицы data на наличие пропусков

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [4]:
len(data[data['days_employed'].isna()]) #подсчёт кол-ва пропусков 

2174

Получим уникальные значения 

In [5]:
data['children'].sort_values().unique() # получение уникальных значений  в столбце "children" таблицы data 

array([-1,  0,  1,  2,  3,  4,  5, 20])

In [6]:
data[data['children'] == -1]['dob_years'].count() # получение информации о кол-ве некорректного значения в столбце "children" таблицы data

47

In [7]:
data['dob_years'].sort_values().unique() # получение уникальных значений  в столбце "ob_years" таблицы data

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

In [8]:
data[data['dob_years'] == 0]['dob_years'].count() # получение информации о кол-ве некорректного значения в столбце "dob_years" таблицы data

101

In [9]:
data['education'].sort_values().unique() # получение уникальных значений  в столбце "education" таблицы data

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

In [10]:
data['education_id'].sort_values().unique() # получение уникальных значений  в столбце "education_id" таблицы data

array([0, 1, 2, 3, 4])

In [11]:
data['family_status'].sort_values().unique() # получение уникальных значений  в столбце "family_status" таблицы data

array(['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], dtype=object)

In [12]:
data['family_status_id'].sort_values().unique() # получение уникальных значений  в столбце "family_status_id" таблицы data

array([0, 1, 2, 3, 4])

In [13]:
data['gender'].sort_values().unique() # получение уникальных значений  в столбце "gender" таблицы data

array(['F', 'M', 'XNA'], dtype=object)

In [14]:
data[data['gender'] == 'XNA']['dob_years'].count() # получение информации о кол-ве некорректного значения в столбце "gender" таблицы data

1

In [15]:
data['income_type'].sort_values().unique() # получение уникальных значений  в столбце "income_type" таблицы data

array(['безработный', 'в декрете', 'госслужащий', 'компаньон',
       'пенсионер', 'предприниматель', 'сотрудник', 'студент'],
      dtype=object)

In [16]:
data['debt'].sort_values().unique() # получение уникальных значений  в столбце "debt" таблицы data

array([0, 1])

In [17]:
data['purpose'].sort_values().unique() # получение уникальных значений  в столбце "purpose" таблицы data

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

**Вывод**



Изучив таблицу, стоить отметить, что:
1. В столбце `children`имеется случайная ошибка в данных (значение "-1" веротянее всего имелось в виду "1" и 20 - печатная ошибка "2");
2. В столбце `days_employed` имеется ошибка в данных (у "Пенсионеров" завышены значения, вероятно проблема в формате дни/часы, остальные значения с минусом (тире) на мой взгляд ошибка выгрузки и веротяно это положительное значение );
3. В столбце `dob_years` встречается 101 пропуск с нулевым значением, на мой взгляд ошибка неслучайная, возраст не заполнен намеренно ;
4. В столбце `education` разное написание значений (строчным и прописными буквами, а также все значения введены по разному нет общего критерия выбора), необходимо привести всё к одному регистру и объединить в общую структкуру по критериям;
5. Столбец `gender` имеет 1 пропуск "XNA";
6. В столбце `total_income` как и `days_employed` имеются пропуски NaN в одинх и тех же местах, веротяно это случайная ошибка выгрузки, т.к. без этих данных невозможно получить кредит.
6. Столбец `purpose` требует лемматизации (т.к.разнятся формулировки данных).


Предварительно можно полагать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [18]:
data1 = deepcopy(data) #сохранение файла с новым именем для дальнейшней работы

**`children`**

В столбце `children` явно имеелось в виду, что  "-1" это "1" и  "20" детей тоже опечатка меняю на "2". 

In [19]:
data1['children'] = data1['children'].replace(-1,1) # замена некорректного значения "-1" на "1"
data1[data1['children'] == -1]['dob_years'].count() # проверка на наличие значений "-1" после замены
data1['children'] = data1['children'].replace(20,2) # замена некорректного значения "20" на "2"
data1[data1['children'] == 20]['dob_years'].count()# проверка на наличие значений "20" после замены

0

**`gender`**

Сразу же перейду к столбцу `gender`, выгрузим всю строку для изучения информации

In [20]:
data1[data1['gender'] == 'XNA'].head(10) # выгрузка строки со значением 'XNA'

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Возраст, образование, стаж работы, наличие гражданского брака и цель кредита скорее всего говорит о том, что это мужской пол.


In [21]:
data1.loc[10701, 'gender'] = 'M' # обращение к элемнту по номеру строки и замена на мужской пол

data1.loc[10701, 'gender'] # проверка значения после замены

'M'

**`days_employed`**

Приведем значения к корректным.
Минусовые значения преобразовываем в положительные, 
завышенные значения "пенсионеров" переведем в дни 
делением на 24 ч. Т.к. вероятнее всего данные предоставлены в часах.

In [22]:
def item_days_employed(row): # создадние функции 
    if row > 0:
        row = row / 24
        return row
    elif row < 0:
        row = row * -1
        return row
    else:
        return row
    
data1['days_employed'] = data1['days_employed'].apply(item_days_employed) # замена значений на корректные 
data1.head(10) # вывод первых 10 строк после замены  таблицы data1



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**`days_employed` и `total_income`**

Пропущенных значений в этих столбцах около 20 % их необходимо заменить. На основе предоставленных данных считаю, что заполнить их более корректно можно опираясь на данные столбца income_type. 
Выгрузим уникальные значения столбца `income_type`


In [23]:
data1[data1['days_employed'].isna()]['income_type'].unique() # выгрузка уникальных значений столбца ncome_type

array(['пенсионер', 'госслужащий', 'компаньон', 'сотрудник',
       'предприниматель'], dtype=object)

Проверим максимальное и минимальное значения столбца `days_employed`

In [24]:
data1['days_employed'].min()

24.14163324048118

In [25]:
data1['days_employed'].max()

18388.949900568383

Т.к. Минимальное и Максимальное значений столбца `days_employed` сильно различаются, считаю некорретным использование среднего значения для заполнения пропусков. Поэтому воспользуюсь медианой.

Рассчитаем медиану для столбца `days_employed` по каждому типу занятости для этого создадим функцию

In [26]:
def mean_and_median(row): 
    data_median = row.median()
    return data_median
   
# employee - сотрудник
# pensioner - пенсионер
# state_employee - госслужащий
# companion - компаньон
# businessman - предприниматель
    
employee_days = data1['days_employed'][data1['income_type'] == 'сотрудник']    
pensioner_days = data1['days_employed'][data1['income_type'] == 'пенсионер']
state_employee_days = data1['days_employed'][data1['income_type'] == 'госслужащий']
companion_days = data1['days_employed'][data1['income_type'] == 'компаньон']
businessman_days = data1['days_employed'][data1['income_type'] == 'предприниматель']
print('сотрудник', mean_and_median(employee_days)) # выведем полученные значения по каждому типу
print('пенсионер', mean_and_median(pensioner_days))
print('госслужащий', mean_and_median(state_employee_days))
print('компаньон', mean_and_median(companion_days))
print('предприниматель', mean_and_median(businessman_days)) 

сотрудник 1574.2028211070851
пенсионер 15217.221094405466
госслужащий 2689.3683533043886
компаньон 1547.3822226779334
предприниматель 520.8480834953765


Рассчитаем медиану для столбца `total_income` по каждому типу занятости для этого создадим функцию

In [27]:
def mean_and_median(row):
    data_median = row.median()
    return data_median
   

# employee - сотрудник
# pensioner - пенсионер
# state_employee - госслужащий 
# companion - компаньон
# businessman - предприниматель
    
employee_income = data1['total_income'][data1['income_type'] == 'сотрудник']    
pensioner_income = data1['total_income'][data1['income_type'] == 'пенсионер']
state_employee_income = data1['total_income'][data1['income_type'] == 'госслужащий']
companion_income = data1['total_income'][data1['income_type'] == 'компаньон']
businessman_income = data1['total_income'][data1['income_type'] == 'предприниматель']
print('сотрудник', mean_and_median(employee_income))  # выведем полученные значения по каждому типу
print('пенсионер', mean_and_median(pensioner_income))
print('госслужащий', mean_and_median(state_employee_income))
print('компаньон', mean_and_median(companion_income))
print('предприниматель', mean_and_median(businessman_income))


сотрудник 142594.39684740017
пенсионер 118514.48641164352
госслужащий 150447.9352830068
компаньон 172357.95096577113
предприниматель 499163.1449470857


Соберем данные в словарь для упрощения дальнейшего заполнения пропусков в столбцах `days_employed` и `total_income`

In [28]:
dictionary = {'сотрудник': [mean_and_median(employee_days), mean_and_median(employee_income)],
            'пенсионер': [mean_and_median(pensioner_days), mean_and_median(pensioner_income)],
             'госслужащий': [mean_and_median(state_employee_days), mean_and_median(state_employee_income)],
             'компаньон': [mean_and_median(companion_days), mean_and_median(companion_income)],
             'предприниматель': [mean_and_median(businessman_days), mean_and_median(businessman_income)]}
                                 
                                 
print(dictionary)

{'сотрудник': [1574.2028211070851, 142594.39684740017], 'пенсионер': [15217.221094405466, 118514.48641164352], 'госслужащий': [2689.3683533043886, 150447.9352830068], 'компаньон': [1547.3822226779334, 172357.95096577113], 'предприниматель': [520.8480834953765, 499163.1449470857]}


Заменяем пропуски на 0 , а заатем на полученные значения из словаря

In [29]:
data1['days_employed'] = data1['days_employed'].fillna(0) # замена пропусков на 0
data1['total_income'] = data1['total_income'].fillna(0)

def days_and_income(row, dictionary): # создание функции для замены 0 на медианы значений по типу
    if row[1] == 0:
        row[1] = dictionary[row[8]][0]
        row[10] = dictionary[row[8]][1]
    return row

data1 = data1.apply(days_and_income, axis=1, args=[dictionary]) 
print(data1.info()) # Вывод на экран общей информации по таблице data1 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


Как видно от пропущенных значений избавились

**`dob_years`**

Можно было бы заполнить возраст по стажу работу, но они сильно разняться по цифрам, плюс не всегда стаж работы начинается в 18 лет. Т.к. столбец с возрастом имеет нормальное распределение - заменю 0 в `dob_years` на средние значения, к тому же это не важно для нашего исследования и проверки гипотез.

In [30]:
data1['dob_years'] = data1['dob_years'].replace(0, int(data1['dob_years'][data1['dob_years'] != 0].mean())) 
data1['dob_years'][data1['dob_years'] == 0].count() # замена 0 на ср.значения и проверка после замены

0

**Вывод**

После предобработки данных было исправлено:
* `children` — заменены некорректные значения;
* `gender` — устранен недочёт
* `dob_years` — нулевые значения устранены
* `total_income` как и `days_employed` —  замнены NaN на медианные значения.

Теперь данные готовы для дальнейшней обработки и исследования. 

### Замена типа данных

Необходимо заменить типа данных в столбцах float на Int в столбцах `total_income` как и `days_employed`.
На итоговый результат дробная часть не повлияет.

In [31]:
data1['days_employed'] = data1['days_employed'].astype('int') # замена типа данных int
data1['total_income'] = data1['total_income'].astype('int')
data1.info() # вывод общей информации для проверки после замены типа данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Тип данных изменен на целые числа int это поможет упростить дальнейшее исследование и избежать ошибок в расчетах.

### Обработка дубликатов

После проверки данных мы выяснили, что у нас имеются как категориальные  так и количественные данные.
Начнем с поиска дубликатов в категориальных данных.
Из подгурженной выше информации видно, что столбец 'education' нуждается в переводе значений к нижнему регистру.

In [32]:
data1['education'] = data1['education'].str.lower() # перевод значений к нижнему регистру
data1['education'].unique() # поиск уникальных значений после замены
data1.groupby('education')['education'].count() # вывод на экран 

education
высшее                  5260
начальное                282
неоконченное высшее      744
среднее                15233
ученая степень             6
Name: education, dtype: int64

Далее произведем поиск дубликатов.

In [33]:
data1.duplicated().sum()

71

In [34]:
data1 = data1.drop_duplicates() # удаление дубликатов
data1.duplicated().sum() # проверка на наличие дубликатов после удаления

0

**Вывод**

Выявленные дубликаты удалены и не помешают дальнейшему исследованию.

### Лемматизация

Для ответа на один из вопросов нам понадобится выявить основные цели взятия кредита. Используем лемматизацию столбца `purpose`.

In [35]:
data1['purpose'].value_counts() # проверка кол-ва всех значений в данном столбце 'purpose'

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [36]:
purpose_unique = data1['purpose'].unique() # определение уникальных значений в столбце 'purpose'
purpose_unique = (" ".join(purpose_unique)) # объединение значений в список
lemmas = m.lemmatize(purpose_unique) # применение метода lemmatize к списку
lemmas = " ".join(set(lemmas)) # объединим леммы в список
lemmas = sorted(list(lemmas.split())) # сортировка списка в алфавитном порядке
lemmas # вывод на экран 

['автомобиль',
 'высокий',
 'для',
 'дополнительный',
 'жилой',
 'жилье',
 'заниматься',
 'коммерческий',
 'на',
 'недвижимость',
 'образование',
 'операция',
 'подержать',
 'покупка',
 'получение',
 'приобретение',
 'проведение',
 'профильный',
 'ремонт',
 'с',
 'свадьба',
 'свой',
 'сдача',
 'сделка',
 'семья',
 'со',
 'собственный',
 'строительство',
 'сыграть']

**Вывод**

Из полученных данных можно определить несколько основных позиций :

* `образование` — расходы на получение образования;
* `свадьба` — расходы связанные с проведением и организацией свадьбы;
* `автомобиль` — расходы на авто, приобретение;
* `жилая недвижимость` —  жилая недвижимость, ремонт также можно отнести к жилью.
* `коммерческая недвижимость ` -  строительство и сдача недвижимости.

Лемматизация поможет категоризовать цель кредита. 


### Категоризация данных

Для ответа на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?" необходимо категоризовать данные по столбцу `children`. 
* Разделим на категории по кол-ву детей :
* `бездетная семья`- категория лиц не имеющих детей;
* `малая семья` - семья с двумя и менее детьми;
* `многодетная семья` - семья с тремя и более детьми.


In [37]:
def categorise_children(row): # создание функции в зависимости от значения в ячейке передаст определние семьи
    if row == 0:
        return 'бездетная семья'
    elif row < 3:
        return 'малая семья'
    else:
        return 'многодетная семья'
    
data1['categorise_children'] = data1['children'].apply(categorise_children) # создание нового столбца с полученными значениями
data1.head(10) # вывод на экран 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,categorise_children
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,малая семья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,малая семья
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,бездетная семья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,многодетная семья
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,бездетная семья
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,бездетная семья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,бездетная семья
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,бездетная семья
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,малая семья
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,бездетная семья


In [38]:
data1['categorise_children'].value_counts() # вывод кол-ва строк по каждому значению

бездетная семья      14091
малая семья           6983
многодетная семья      380
Name: categorise_children, dtype: int64

Для ответа на вопрос "Есть зависимость между уровнем дохода и возвратом кредита в срок" категоризуем данные по столбцу `total_income`.


In [39]:
data1['total_income'].median() # рассчитаем медиану столбца "total_income"

142594.0

Исходя из этого определим пределы значений с шагом 50 т.р.

In [40]:
def total_inc_categories(row): # создание функции, исходя от значения дохода определит категорию 
    if row < 100000:
        return 'Доход до 100 тысяч'
    elif row < 150000:
        return 'Доход от 100 до 150 тысяч'
    elif row < 200000:
        return 'Доход от 150 до 200 тысяч'
    elif row < 250000:
        return 'Доход от 200 до 250 тысяч'   
    else:
        return 'Доход от 250 тысяч'
    
data1['total_inc_categories'] = data1['total_income'].apply(total_inc_categories) # создание нового столбца с занесением новых значений
display(data1) # вывод на экран  
data1['total_inc_categories'].value_counts() # вывод кол-ва строк по каждому значению

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,categorise_children,total_inc_categories
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,малая семья,Доход от 250 тысяч
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,малая семья,Доход от 100 до 150 тысяч
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,бездетная семья,Доход от 100 до 150 тысяч
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,многодетная семья,Доход от 250 тысяч
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,бездетная семья,Доход от 150 до 200 тысяч
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,малая семья,Доход от 200 до 250 тысяч
21521,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,бездетная семья,Доход от 150 до 200 тысяч
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,малая семья,Доход до 100 тысяч
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,многодетная семья,Доход от 200 до 250 тысяч


Доход от 100 до 150 тысяч    7160
Доход от 150 до 200 тысяч    4764
Доход до 100 тысяч           4463
Доход от 250 тысяч           2813
Доход от 200 до 250 тысяч    2254
Name: total_inc_categories, dtype: int64

Также создадим категоризацию данных для столбца `purpose` т.к. цель кредита важна для исследования.

Создаздим через словарь категории целей кредита и добавим в таблицу новый столбец. 


In [41]:
purpose_dictionary = {'свадьба':0, 'автомобиль':1, 'образование':2, 'жилой':3, 'жилье':3, 'собственный':3,  'коммерческий':4, 'недвижимость':4}


def set_purpose_category(row, purpose_dictionary): # создание функции, которая вернет категорию целей кредита
    lemma = m.lemmatize(row)
    for word in lemma:
        if word in purpose_dictionary:
            return purpose_dictionary[word]
        

data1['purpose_category'] = data1['purpose'].apply(set_purpose_category, args=[purpose_dictionary]) # создание нового столбца с добавлением полученных значений
data1['purpose_category'].value_counts()   # вывод кол-ва строк по каждому значению


3    6325
4    4486
1    4306
2    4013
0    2324
Name: purpose_category, dtype: int64

In [42]:
purpose_dictionary1 = {0: 'свадьба', 1: 'автомобиль', 2: 'образование', 3: 'жилая недвижимость', 4: 'коммерческая недвижимость'}

def category_purpose(row):   # создание функции, которая вернет по числу категорию
    return purpose_dictionary1[row]

data1['purpose_category'] = data1['purpose_category'].apply(category_purpose)
data1['purpose_category'].value_counts() # вывод кол-ва строк по каждому значению

жилая недвижимость           6325
коммерческая недвижимость    4486
автомобиль                   4306
образование                  4013
свадьба                      2324
Name: purpose_category, dtype: int64

**Вывод**

Произведена категоризация данных для столбца `children`, `purpose`, `total_income`.
Теперь данные готовы для проверки гипотез.

## Шаг 3. Ответьте на вопросы

**- Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [43]:
data1.rename(columns={'categorise_children': 'Категория'}, inplace=True) # вывод данных в таблицу
table_1 = pd.pivot_table(data1, index='Категория', values='debt', aggfunc='count') # создание таблицы
table_2 = pd.pivot_table(data1, index='Категория', values='debt', aggfunc='sum') 
child_dept = table_1
child_dept['not_return'] = table_2['debt']
child_dept['returned'] = child_dept['debt'] - child_dept['not_return'] # расчёт показателей
child_dept['return_percent'] = child_dept['returned'] / child_dept['debt']
child_dept['return_percent'] = child_dept['return_percent'].mul(100).round(1).astype(str).add('%')
child_dept = child_dept.sort_values(by='return_percent', ascending=False) # сортировка по убыванию
child_dept.rename(columns={'debt': 'Всего', 'not_return': 'Не выплачено', 'returned': 'Выплачено', 'return_percent': 'Процент выплаченных кредитов'}, inplace=True) # переименовка столбцов на русский язык
child_dept # вывод на экран

,Всего,Не выплачено,Выплачено,Процент выплаченных кредитов
Категория,,,,
бездетная семья,14091,1063,13028,92.5%
многодетная семья,380,31,349,91.8%
малая семья,6983,647,6336,90.7%


**Вывод**

Исходя из полученных данных, можно сказать о том, что кол-во детей слабо влияет на возвраст кредита в срок.
* На первом месте семьи без детей, скорее всего это связано с тем, что у таких семей нет непредвиденных и спонтанных затрат , а также затрат на содержание детей, поэтому таким семьям легче спрогнозировать свои затраты.
* На втором месте многодетные семьи,полагаю что такие семьи подходят более ответственно к затрам в семье, плюс субсидий у них больше нежели у малодетных семей - это упрощает выплату кредитов.
* На третьем месте малые семьи - это семьи с одним или двумя детьми, скорее всего самый низкий процент выплаченных кредитов говорит о том, что семьи еще не умеют четко прогнозировать свои финанс и часть кредитов берут с переоценкой своих возможностей.Также между содержанием одного и двух детей есть существенная разница в сравнении с многодетными семьями и возможно на момент взятия кредита у таких семей был только один ребенок, а второй появился в последствии. 

**- Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [44]:
data1.rename(columns={'family_status': 'Семейный статус'}, inplace=True) # вывод данных в таблицу
table_1 = pd.pivot_table(data1, index='Семейный статус', values='debt', aggfunc='count') # создание таблицы
table_2 = pd.pivot_table(data1, index='Семейный статус', values='debt', aggfunc='sum')
family_status_dept = table_1
family_status_dept['not_return_debt'] = table_2['debt']
family_status_dept['returned_debt'] = family_status_dept['debt'] - family_status_dept['not_return_debt']
family_status_dept['return_percent'] = family_status_dept['returned_debt'] / family_status_dept['debt']
family_status_dept['return_percent'] = family_status_dept['return_percent'].mul(100).round(1).astype(str).add('%')
family_status_dept = family_status_dept.sort_values(by='return_percent', ascending=False)  # сортировка по убыванию
family_status_dept.rename(columns={'debt': 'Всего', 'not_return_debt': 'Не выплачено', 'returned_debt': 'Выплачено', 'return_percent': 'Процент выплаченных кредитов'}, inplace=True)
family_status_dept # вывод на экран

,Всего,Не выплачено,Выплачено,Процент выплаченных кредитов
Семейный статус,,,,
вдовец / вдова,959,63,896,93.4%
в разводе,1195,85,1110,92.9%
женат / замужем,12339,931,11408,92.5%
гражданский брак,4151,388,3763,90.7%
Не женат / не замужем,2810,274,2536,90.2%


**Вывод**

Основываясь на полученных данных, можно сказать о том, что у всех категорий процент превышает планку в 90%.
* Однако на первом месте те, кто имеют семейный статус "вдовец / вдова", скорее всего это связано с тем, что у таких людей большой жизненный опыт и достаточно накополений, а также  они сами несут отвественность за свои финансы.
* Далее те, кто в разводе, это также связано с тем, что такие люди сами отвественны за свои финансы и траты.
* На третьем месте категория "женат/замужем" при взятии кредита имеют понимание, что собираются жить вместе и расширять свою семью, поэтому несут ответственность за погашение кредита в срок.
* Самые низкие показатели имеют категории "гражданский брак" и "Не женат / не замужем" не несут полной ответственности и могут иметь необдуманные траты.

**- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [45]:
data1.rename(columns={'total_inc_categories': 'Категория дохода'}, inplace=True)
table_1 = pd.pivot_table(data1, index='Категория дохода', values='debt', aggfunc='count')
table_2 = pd.pivot_table(data1, index='Категория дохода', values='debt', aggfunc='sum')
total_income_dept = table_1
total_income_dept['not_return_debt'] = table_2['debt']
total_income_dept['returned_debt'] = total_income_dept['debt'] - total_income_dept['not_return_debt']
total_income_dept['return_percent'] = total_income_dept['returned_debt'] / total_income_dept['debt']
total_income_dept['return_percent'] = total_income_dept['return_percent'].mul(100).round(1).astype(str).add('%')
total_income_dept = total_income_dept.sort_values(by='return_percent', ascending=False) 
total_income_dept.rename(columns={'debt': 'Всего', 'not_return_debt': 'Не выплачено', 'returned_debt': 'Выплачено', 'return_percent': 'Процент выплаченных кредитов'}, inplace=True)
total_income_dept

,Всего,Не выплачено,Выплачено,Процент выплаченных кредитов
Категория дохода,,,,
Доход от 250 тысяч,2813,194,2619,93.1%
Доход от 200 до 250 тысяч,2254,164,2090,92.7%
Доход до 100 тысяч,4463,354,4109,92.1%
Доход от 150 до 200 тысяч,4764,405,4359,91.5%
Доход от 100 до 150 тысяч,7160,624,6536,91.3%


**Вывод**

* Самый высокий процент имеет категория "Доход от 250 тысяч" это ожидаемо т.к. чем больше заработок тем больше свободын ресурсов на оплату кредита.
* За ним расположилась категория Доход от 200 до 250 тысяч по тем же причинам.
* Далее "Доход до 100 тысяч" веротяно это связано с отвественностью и распределением своего зароботка на все нужды.
* Категории "Доход от 100 до 150 тысяч" и "Доход от 150 до 200 тысяч" имеют приблизительно одинаковую поизицю, веротяно связано с тем, что люди перешли на новый обрах жизни, в котором и расходы больше. Может так быть связано с наличем у такцих лиц нескольких кредитов сразу.



**- Как разные цели кредита влияют на его возврат в срок?**

In [46]:
data1.rename(columns={'purpose_category': 'Цель кредита'}, inplace=True)
table_1 = pd.pivot_table(data1, index='Цель кредита', values='debt', aggfunc='count')
table_2 = pd.pivot_table(data1, index='Цель кредита', values='debt', aggfunc='sum')
dept_by_purpose = table_1
dept_by_purpose['not_return_debt'] = table_2['debt']
dept_by_purpose['returned_debt'] = dept_by_purpose['debt'] - dept_by_purpose['not_return_debt']
dept_by_purpose['return_percent'] = dept_by_purpose['returned_debt'] / dept_by_purpose['debt']
dept_by_purpose['return_percent'] = dept_by_purpose['return_percent'].mul(100).round(1).astype(str).add('%')
dept_by_purpose = dept_by_purpose.sort_values(by='return_percent', ascending=False) 
dept_by_purpose.rename(columns={'debt': 'Всего', 'not_return_debt': 'Не выплачено', 'returned_debt': 'Выплачено', 'return_percent': 'Процент выплаченных кредитов'}, inplace=True)
dept_by_purpose

,Всего,Не выплачено,Выплачено,Процент выплаченных кредитов
Цель кредита,,,,
жилая недвижимость,6325,439,5886,93.1%
коммерческая недвижимость,4486,343,4143,92.4%
свадьба,2324,186,2138,92.0%
образование,4013,370,3643,90.8%
автомобиль,4306,403,3903,90.6%


**Вывод**

* Самый высокий показатель имеет цель - "жилая недвижимость" это очевидно, т.к. это играет большую роль в социальной жизни. Приобретая недвижимость в кредит, заемщики учитывают риски и подходят к выбору очень осознанно и отвественно.
* "Коммерческая недвижимость" как способ зароботка распологается на втором месте, здесь также учитываются многие факторы в том числе и затем полученная прибыль для дальнейшего закрытия кредита.
* "Свадьба" находится на третьем месте, обычно это не такие большие деньги в кредит как, недвижимость, поэтому здесь и выплатить кредит легче, особенно с учетом прибыли от денежных презентов на мероприятии.
* "Образование" и "Автомобиль" можно объединить по критерию возврата т.к. здесь часто возникают непредвиданные риски.  В "Образовании" например - отчисление, потеря мотивации и т.д. В "Автомобиле" различные затраты на ремонт, страховку, ДТП и прочее.

## Шаг 4. Общий вывод

После проведения необходимых исследований, в целом ситуацию можно описать как благоприятную.Во всех пункатх исследования возврат кредита имеет показатель более 90 %. 
* Наличие детей иммет самый маленький разброс по процентам, поэтому здесь заивимость слабо выражена. Но в общем можно сделать вывод, что самый низкий процентов по возврату имеют малые семьи с один и двумя детьми.Поэтому при выдаче кредита этот критерий из 4х исследуемых я бы отнесла на последнюю позицию.
* Зависимость между семейным положением и возврата кредита в срок имеет самый большй разброс показателей, поэтому необходимо обращать на него особое внимание при выдаче кредита. В целом три категории имеют самый больший процент выплаченных кредитов вдовец / вдова, в разводе, женат / замужем.
* Зависимость от показателя "уровень дохода" определяется неоднозначно, с одной стороны те, кто имеет высокий зароботок, с другой те кто имеют средний заработок. Не самый решающий показатель.
* Цель кредита имеет второй по величине разброс показателей процента выплаченных кредитов. Здесь на первом месте расположилась недвижимость.

* В заключение основываясь на полученных данных идельным кандидатом для выдачи кредита будет человек с высшим образованием, высоким или средним зароботком, без детей, в разводе и целью его кредита будет недвижимость как жилая так и коммерческая.


